In [1]:
import numpy as np
import pandas as pd
import os
from collections import OrderedDict

In [2]:
from platform import python_version
print(python_version())

3.6.6


### Exp of classifing movie genres based on their events.


In [3]:
movies = pd.read_csv('../../data/movie_metadata_cleaned.tsv', sep = '\t')

In [4]:
movies.head(2)

,Wikipedia_id,Freebase_id,Name,Release_date,Revenue,Runtime,Languages,Countries,Genres
0,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,dict_values(['English Language']),dict_values(['United States of America']),Drama
1,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,dict_values(['Norwegian Language']),dict_values(['Norway']),Drama


### Read in events data

In [5]:
event_file = [item for item in os.listdir('../../data/protag_events_v2/') if 'txt' in item]

In [16]:
d = {}
for name in event_file:
    with open(os.path.join('../../data/protag_events_v2/', name), 'r') as f:
        #only use the verbs
        verbs = []
        for line in [line.strip() for line in f.readlines()]:
            line = line.split(' ')
            verbs.append(line[1])
        if len(verbs) > 1:    
            d[int(name.split('_')[0])] = verbs

In [7]:
d = {}
for name in event_file:
    with open(os.path.join('../../data/protag_events_v2/', name), 'r') as f:
        #only use the verbs
        nlines = []
        for line in [line.strip() for line in f.readlines()]:
            line = line.split(' ')
            line[0] = 'protagonist'
            line = ' '.join([item for item in line[0:2]])
            nlines.append(line)
        if len(nlines) > 1:    
            d[int(name.split('_')[0])] = nlines

In [17]:
movies.Wikipedia_id = movies.Wikipedia_id.astype(int)

In [18]:
movies['Events'] = movies['Wikipedia_id'].map(d)

In [19]:
movies = movies.dropna(subset = ['Events'])

In [20]:
movies['Genre_code'] = movies.Genres.astype("category").cat.codes

In [21]:
dict( enumerate(movies.Genres.astype("category").cat.categories) )

{0: 'Action', 1: 'Comedy', 2: 'Drama', 3: 'Romance Film', 4: 'Thriller'}

In [22]:
movies.head(2)

,Wikipedia_id,Freebase_id,Name,Release_date,Revenue,Runtime,Languages,Countries,Genres,Events,Genre_code
3,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,dict_values(['German Language']),dict_values(['Germany']),Drama,"[force-59, learn-14-1, contiguous_location-47....",2
10,11250635,/m/02r52hc,The Mechanical Monsters,NaN,NaN,NaN,dict_values(['English Language']),dict_values(['United States of America']),Action,"[weather-57, battle-36.4-1, require-103-1, see...",0


In [23]:
movies.Genre_code.value_counts()

2    2419
1    1643
4     679
0     485
3     144
Name: Genre_code, dtype: int64

In [24]:
movies.to_csv('../../data/metadata_events_table.tsv', sep = '\t', index=False)